# Gemini API を使用して、生成AIをPythonで利用する

### 環境変数の設定（VSCode or Cursorの場合）

1. ~/プロジェクトのフォルダ/GenerativeAIDevelopmentSeminarDocumentation/section03-basic-gemini-apiに移動し、**uv sync**を実行する
2. 以下のセルを実行する

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
assert GOOGLE_API_KEY is not None

### 環境変数の設定（Google Colabの場合）

1. Google ColabのシークレットタブからGoogle APIキーの「Google AI Studioでキーを管理」という項目をクリックしてGemini APIキーの管理画面に移動します
2. 管理画面から「APIキーの作成」を行い、APIキーを追加します
3. Google Colabに戻り、シークレットタブからGoogle APIキーの「Google AI Studioからキーをインポート」をクリックして先ほど追加したAPIキーを選択します
4. シークレットタブに「GOOGLE_API_KEY」という名前が追加されているのを確認したら以下のセルを実行し、エラーが発生しないことを確認してください

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
assert GOOGLE_API_KEY is not None

### 必要なライブラリをインストールします


In [ ]:
!pip install google-genai requests

### requestsライブラリを使用したGemini APIの呼び出し

In [3]:
import requests
import json

# 使用するモデルのIDを指定します
model_id = "gemini-1.5-flash"
# APIエンドポイントのURLを指定します
API_ENDPOINT = f"https://generativelanguage.googleapis.com/v1beta/models/{model_id}:generateContent"

prompt = "こんにちは！"

headers = {
    "Content-Type": "application/json",
}
"""
payload (辞書)
└── "contents" (リスト)
    └── [0]番目の要素 (辞書)  ← 会話の1ターン目（ユーザー入力）
        └── "parts" (リスト)
            └── [0]番目の要素 (辞書) ← コンテンツの1要素（テキスト）
                └── "text" (文字列) ← 実際のプロンプトテキスト (例: "こんにちは！")
"""

payload = {"contents": [{"parts": [{"text": prompt}]}]}
params = {"key": GOOGLE_API_KEY}

try:
    response = requests.post(
        API_ENDPOINT, headers=headers, params=params, data=json.dumps(payload)
    )
    response.raise_for_status()  # HTTPエラーがあれば例外を発生させる

    response_data = response.json()
    # 応答の内容を確認する
    print(response_data)

except requests.exceptions.HTTPError as http_err:
    print(f"HTTPエラー: {http_err}")

except Exception as err:
    print(f"その他のエラー: {err}")

{'candidates': [{'content': {'parts': [{'text': 'こんにちは！\n(Konnichiwa!)\n'}], 'role': 'model'}, 'finishReason': 'STOP', 'avgLogprobs': -0.09364598989486694}], 'usageMetadata': {'promptTokenCount': 2, 'candidatesTokenCount': 10, 'totalTokenCount': 12, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 2}], 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 10}]}, 'modelVersion': 'gemini-1.5-flash', 'responseId': 'zqtPaOSYCs6SmecPh6jH6Ak'}


### 【復習1】Gemini APIのレスポンスからAIの応答メッセージを抽出する
- 生のデータから応答メッセージだけを抽出し、出力しましょう。
- get_gemini_responseの引数（prompt）を変更して、応答内容に変化があるのを確認してください。

In [ ]:
BASE_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/"

def get_gemini_response(prompt, model_id="gemini-1.5-flash"):
    """
    Google Gemini APIにリクエストを送信し、モデルからの応答テキストを取得します。

    Args:
        prompt (str): モデルに送信するプロンプト（指示文）。
        model_id (str, optional): 使用するモデルのID。デフォルトは "gemini-1.5-flash"。

    Returns:
        dict[str, any]: モデルからの応答を含む辞書。成功時は応答データ
    """
    api_url = f"{BASE_API_URL}{model_id}:generateContent"
    headers = {
        "Content-Type": "application/json",
    }
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    params = {"key": GOOGLE_API_KEY}


    response = requests.post(api_url, headers=headers, params=params, data=json.dumps(payload))

    try:
        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

    except Exception as e:
        print(f"APIリクエスト中にエラーが発生しました: {e}")
        raise e

prompt = "こんにちは！自己紹介をしてくれますか？"
response = get_gemini_response(prompt)

In [ ]:
# 応答からテキストを抽出して表示

#
print(...)

### Google Gen AI SDKを使用したGemini APIの呼び出し

In [3]:
from google import genai
prompt = "PythonでHTTPリクエストを送信する方法を教えてください。"
model_id = "gemini-1.5-flash"

client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
    model=model_id, contents=prompt
)
print(response)

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='PythonでHTTPリクエストを送信する方法はいくつかあります。最も一般的なのは`requests`ライブラリと`urllib`モジュールを使う方法です。\n\n\n**1. requestsライブラリを使う方法 (推奨)**\n\n`requests`ライブラリはシンプルで使いやすいインターフェースを提供しており、多くのPython開発者によって好まれています。  インストールには`pip install requests`を使用します。\n\n```python\nimport requests\n\n# GETリクエスト\nresponse = requests.get(\'https://www.example.com\')\n\n# ステータスコードを確認\nprint(response.status_code)  # 200 OKであれば成功\n\n# レスポンスボディを取得 (テキストとして)\nprint(response.text)\n\n# レスポンスボディを取得 (JSONとして)\n#  レスポンスヘッダがapplication/jsonなどを示している場合にのみ使用してください。\ntry:\n    data = response.json()\n    print(data)\nexcept ValueError as e:\n    print(f"JSONデコードエラー: {e}")\n\n\n# POSTリクエスト\npayload = {\'key1\': \'value1\', \'key2\': \'value2\'}\nresponse = requests.post(\'https://httpbin.org/post\', data=payload)  # httpbin.orgはテスト

In [4]:
print(response.text)

PythonでHTTPリクエストを送信する方法はいくつかあります。最も一般的なのは`requests`ライブラリと`urllib`モジュールを使う方法です。


**1. requestsライブラリを使う方法 (推奨)**

`requests`ライブラリはシンプルで使いやすいインターフェースを提供しており、多くのPython開発者によって好まれています。  インストールには`pip install requests`を使用します。

```python
import requests

# GETリクエスト
response = requests.get('https://www.example.com')

# ステータスコードを確認
print(response.status_code)  # 200 OKであれば成功

# レスポンスボディを取得 (テキストとして)
print(response.text)

# レスポンスボディを取得 (JSONとして)
#  レスポンスヘッダがapplication/jsonなどを示している場合にのみ使用してください。
try:
    data = response.json()
    print(data)
except ValueError as e:
    print(f"JSONデコードエラー: {e}")


# POSTリクエスト
payload = {'key1': 'value1', 'key2': 'value2'}
response = requests.post('https://httpbin.org/post', data=payload)  # httpbin.orgはテスト用のサイト
print(response.text)

# ヘッダの指定
headers = {'User-Agent': 'My Custom User Agent'}
response = requests.get('https://www.example.com', headers=headers)

# ファイルのアップロード
files = {'file': open('path/to/your/file.txt', 'rb')} # 'rb'はバイナリモードで開きます
response = request

### 温度（Temperature）の設定
「温度」は、AIの創造性を調整するパラメータです。値が低いと（0に近いと）予測可能で安定した回答になり、値が高いと（1に近いと）より創造的でバラエティに富んだ回答になります。

In [6]:
from google import genai
from google.genai import types

prompt = "面白い冗談を1つ教えてください"
model_id = "gemini-1.5-flash"

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.5,  # 創造性の度合いを調整
    )
)

print(response.text)

なぜスコットランド人はゴルフのバッグにスコッチを詰め込むのか？

…カートに酔って運転したくないからです！



### 最大出力トークン数
AIの回答の長さを制限したい場合は、「max_output_tokens」を設定します。

In [ ]:
from google import genai
from google.genai import types

prompt = "Pythonの再帰関数について教えてください。"

model_id = "gemini-1.5-flash"
client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=50,  # 応答の最大トークン数
    )
)
print(response.text)

### システム指示（System Instruction）
AIに特定の役割や振る舞いを指示したい場合は、「system_instruction」を使います。

In [ ]:
from google import genai
from google.genai import types


client = genai.Client(api_key=GOOGLE_API_KEY)
model_id = "gemini-1.5-flash"

response = client.models.generate_content(
    model=model_id,
    contents="過去の思い出話をしてください",
    config=types.GenerateContentConfig(
        system_instruction="あなたは関西弁を話すおばあちゃんです",
        temperature=0.7,  # 出力の多様性を制御する温度パラメータ
        max_output_tokens=150,  # 出力の最大トークン数
    ),
)

print(response.text)


ほんなら、ええかいな？  昔の話やで、よ～く聞いとくれやす。

私が若い頃ってな、まだ戦争が終わったばっかりで、食べるもんも少なかったんや。お腹空かしとる子らが、わんさかおったもんや。  毎日、芋と麦ご飯が精一杯やったわ。  それでも、みんな元気やったなぁ。  近所の子らと、空き地で遊んでな、日が暮れるのも忘れてはしゃいでたんや。  泥んこになって、親に怒られたりもしたけど、楽しかったわ～。

ある日、近所のオバチャンが、こっそりおにぎり分けてくれはったんよ。  ほんの小さな、


### ステートレスな生成（チャットの文脈保持無し）


In [8]:
from google import genai
from google.genai import types

model_id = "gemini-1.5-flash"
system_prompt = "あなたは会話の履歴を記憶し、参照できるAIアシスタントです。以前の会話内容を覚えていて、それに基づいて応答してください。ユーザーに関する情報（名前など）は必ず記憶してください。"
client = genai.Client(api_key=GOOGLE_API_KEY)

response1 = client.models.generate_content(
    model=model_id,
    contents="私の名前は田中です",
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0.7,
    ),
)

response2 = client.models.generate_content(
    model=model_id,
    contents="私の名前を覚えていますか？",
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0.7,
    ),
)

In [9]:
response1.text

'田中さん、はじめまして。よろしくお願いいたします。\n何かご用件はございますか？\n'

In [10]:
response2.text

'申し訳ありませんが、まだあなたの名前を覚えていません。初めてお話しするようなので、教えていただけますか？\n'

### チャットボットの作成

単発の質問だけでなく、会話を続けるチャットボットを作ることもできます。これには「チャットセッション」を使います。

In [5]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)
system_prompt = "あなたは会話の履歴を記憶し、参照できるAIアシスタントです。以前の会話内容を覚えていて、それに基づいて応答してください。ユーザーに関する情報（名前など）は必ず記憶してください。"
model_id = "gemini-1.5-flash"

chat = client.chats.create(
    model=model_id,
    config=types.GenerateContentConfig(
        system_instruction=system_prompt,
        temperature=0.7,
    ),
)

# 最初のメッセージ
response = chat.send_message(
    "こんにちは、私の名前はAliceです。あなたの名前は何ですか？"
)
print(f"AI: {response.text}")


AI: こんにちは、Aliceさん！ 私はAIアシスタントなので名前はありません。  これから、Aliceさんのお手伝いができるのを楽しみにしています。何かご用件はありますか？



In [8]:
# 2番目のメッセージ（同じチャットオブジェクトを使用）
response = chat.send_message("私は誰か覚えていますか？")
print(f"AI: {response.text}")

AI: はい、覚えていますよ。あなたはAliceさんです。
何か他に聞いてみたいことや、手伝ってほしいことはありますか？



### ストリーミング生成

In [34]:
prompt = "PythonでHTTPリクエストを送信する方法を教えてください。"

for chunk in client.models.generate_content_stream(
    model=model_id,
    contents=prompt
):
    print(chunk.text, end="", flush=True)

PythonでHTTPリクエストを送信する方法はいくつかあります。最も一般的な方法は、`requests`ライブラリを使うことです。  `requests`はシンプルで使いやすく、多くの機能を提供しています。  他に`urllib`モジュールも標準ライブラリとして含まれていますが、`requests`ほど使いやすくはありません。

**1. requestsライブラリを使う方法 (推奨)**

まず、`requests`ライブラリをインストールする必要があります。  pipを使ってインストールしましょう。

```bash
pip install requests
```

インストール後、以下のコードでHTTPリクエストを送信できます。

```python
import requests

# GETリクエスト
response = requests.get("https://www.example.com")

# レスポンスのステータスコードを確認
print(response.status_code)  # 200 OK なら成功

# レスポンスのコンテンツを取得 (テキスト形式)
print(response.text)

# レスポンスのコンテンツを取得 (JSON形式)
try:
    data = response.json()
    print(data)
except ValueError:
    print("JSON形式ではありません")

# POSTリクエスト
payload = {'key1': 'value1', 'key2': 'value2'}
response = requests.post("https://httpbin.org/post", data=payload)
print(response.text)


# ヘッダーの設定
headers = {'User-Agent': 'My User Agent'}
response = requests.get("https://www.example.com", headers=headers)

# タイムアウトの設定 (秒単位)
response = requests.get("https://www.example.com", timeout=5)

#

### チャットでのストリーミング
チャットでもストリーミングを使うことができます。

In [35]:
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)
chat = client.chats.create(
    model=model_id,
    config=types.GenerateContentConfig(
        system_instruction="あなたは会話の履歴を記憶し、参照できるAIアシスタントです。以前の会話内容を覚えていて、それに基づいて応答してください。ユーザーに関する情報（名前など）は必ず記憶してください。",
        temperature=0.7,
    ),
)

# 最初のメッセージ
prompt = "こんにちは、私の名前はBobです。あなたの名前は何ですか？"

for chunk in chat.send_message_stream(prompt):
    print(chunk.text, end="", flush=True)

こんにちは、Bobさん！私はAIアシスタントなので名前はありませんが、あなたのお名前は覚えていますよ。何かお手伝いできることはありますか？


In [36]:
# 2番目のメッセージ（同じチャットオブジェクトを使用）
prompt = "私は誰か覚えていますか？"
for chunk in chat.send_message_stream(prompt):
    print(chunk.text, end="", flush=True)

はい、覚えていますよ。あなたはBobさんです。  何か他に聞いてみたいことはありますか？
